In [ ]:
%matplotlib notebook

import sys

#Own packages.
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir

    path.append(dir(dir(dir(path[0]))))
    
from src import formatting_helpers
from src import visualisation_helpers

from IPython.core.display import HTML, Math, Markdown
import igraph
import cairo
import pandas as pd
import numpy as np

# Adjacency Matrices for Graph
_<u>Source</u>: P76 WOrked Exampled 2.4A_

In the following examples, we generate 3 graphs.
- **G1** has a bridge between Node 1 and Node 2.
- **G2** removes that bridge.
- **G3** is like **G2**, but with a loop for Node 0.

Observe how the adjanency matrices of **G1**, **G2**, and **G3** change thru various iterations.

In [ ]:
#Helper function. Consider putting in src only if it becomes generalisable.
def plot_graphAndIterations(edge_list, number_of_iterations):
    
    '''
    Params
    edge_list: list of tuples.
    number_of_iterations: positive integer.
    
    Output
    Print
    Return iGraph
    '''
    
    #Create graph object
    G = igraph.Graph.TupleList(edge_list)
    
    #Create labels for vertices in graph object
    for vertex in G.vs:
        vertex['label'] = vertex['name']
    
    G_plot = igraph.plot(G, bbox=(120,120))
    
    display(G_plot)
    
    G_adjMatrix = np.array(G.get_adjacency()._get_data())
    
    for iter_idx in range(1, number_of_iterations+1):
        G_adjMatrixIterated = np.linalg.matrix_power(G_adjMatrix, iter_idx)
        display(HTML('<p><b>Adjacency Matrix after {} Iteration(s)</b></p>'.format(iter_idx)))
        display(Math(formatting_helpers.npMatrix_to_latex(G_adjMatrixIterated)))
    
    return G, G_adjMatrix

### **G1** has a bridge between Node 1 and Node 2

In [ ]:
edge_list1 = pd.DataFrame({'source':[0,0,1,1,2], 
                           'target':[1,2,2,3,3]})
edge_list1_tuples = [tuple(x) for x in edge_list1.values]

G1, G1_adjMatrix = plot_graphAndIterations(edge_list=edge_list1_tuples, number_of_iterations=5)

### **G2** removes that bridge

In [ ]:
edge_list2 = pd.DataFrame({'source':[0,0,1,2], 
                           'target':[1,2,3,3]})
edge_list2_tuples = [tuple(x) for x in edge_list2.values]

G2, G2_adjMatrix = plot_graphAndIterations(edge_list=edge_list2_tuples, number_of_iterations=5)

### **G3** is like **G2**, but with a loop for Node 1

In [ ]:
edge_list3 = pd.DataFrame({'source':[0,0,1,2,0], 
                           'target':[1,2,3,3,0]})
edge_list3_tuples = [tuple(x) for x in edge_list3.values]

G3, G3_adjMatrix = plot_graphAndIterations(edge_list=edge_list3_tuples, number_of_iterations=3)

In [ ]:
row = 0
col = 3

vectors = dict()
for idx, adjMatrix in enumerate([G1_adjMatrix, G2_adjMatrix, G3_adjMatrix]):
    vectors[idx] = {row: adjMatrix[row,:], col: adjMatrix[:,col]}

iteration_number = 2

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import itertools

def plot_previousIterationAdjMatrix(G_adjMatrix, iteration_number, row_number, col_number, ax):
    
    G_previousAdjMatrix = np.linalg.matrix_power(G_adjMatrix, iteration_number-1)
    col_count, row_count = G_previousAdjMatrix.shape
    
    ax.axis('scaled')
    ax.axis('off')
    
    matrix_row_ticks = np.flip(np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], row_count*2+1)[0::2])
    matrix_col_ticks = np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], col_count*2+1)[0::2]
    
    matrix_row_spacing = np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], row_count*2+1)[1::2]
    matrix_col_spacing = np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], col_count*2+1)[1::2]
    matrix_spacing = list(itertools.product(matrix_row_spacing, matrix_col_spacing))
    
    for i, j in zip(G_previousAdjMatrix.flatten(order='C'), matrix_spacing):
        ax.text(j[0], j[1], s=round(i,2),horizontalalignment='center',
                 verticalalignment='center')

    rect_row = patches.Rectangle((matrix_col_ticks[0], matrix_row_ticks[row_number]),
                                 width=matrix_col_ticks[col_count]-matrix_col_ticks[0],
                                 height=matrix_row_ticks[row_number]-matrix_row_ticks[row_number+1],
                                 facecolor='none',
                                 edgecolor='#1f77b4',
                                 linewidth=2,
                                 clip_on=False)
    
    
    rect_col = patches.Rectangle((matrix_col_ticks[col_number], matrix_row_ticks[0]),
                             width=matrix_col_ticks[col_number]-matrix_col_ticks[col_number+1],
                             height=matrix_row_ticks[row_count]-matrix_row_ticks[0],
                             facecolor='none',
                             edgecolor='#ff7f0e',
                             linewidth=2,
                             clip_on=False)
    
    ax.add_patch(rect_row)
    ax.add_patch(rect_col)
    
    return ax

fig, axes_array = plt.subplots(nrows=1, ncols=2)
ax1, ax2 = axes_array

plot_previousIterationAdjMatrix(G2_adjMatrix, iteration_number=5, row_number=3, col_number=2, ax=ax2);
plot_previousIterationAdjMatrix(G3_adjMatrix, iteration_number=4, row_number=1, col_number=1, ax=ax1);
display(igraph.plot(G3, bbox=(120,120)))

In [ ]:
%matplotlib notebook

import matplotlib

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import itertools

from matplotlib.artist import Artist

fig, axes_array = plt.subplots(nrows=3, ncols=2)
(ax1, ax2), (ax3, ax4), (ax5, ax6) = axes_array

def plot_AdjMatrix(G_adjMatrix, iteration_number, ax,row_number=None, col_number=None, cell_idx_list=None):
    
    '''
    Purpose:
    Plot adjacency matrix of previous iteration.
    
    Params:
    row_number: row to box, indexing starting at 0.
    col_number: col to box, indexing starting at 0.
    
    Return:
    ax: Axes object with adjancency matrix plotted and row + col rectangles.
    '''
    
    G_adjMatrix_iterated = np.linalg.matrix_power(G_adjMatrix, iteration_number)
    col_count, row_count = G_adjMatrix_iterated.shape
    display(G_adjMatrix_iterated)
    
    ax.axis('scaled');
    ax.axis('off');
    
    matrix_row_ticks = np.flip(np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], row_count*2+1)[0::2]) #flip because min(ylim) demarcates top row
    matrix_col_ticks = np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], col_count*2+1)[0::2]
    
    matrix_row_spacing = np.flip(np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], row_count*2+1)[1::2])  #flip because min(ylim) demarcates top row
    matrix_col_spacing = np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], col_count*2+1)[1::2]
    matrix_spacing = list(itertools.product(matrix_col_spacing,matrix_row_spacing))
    
    for i, j in zip(G_adjMatrix_iterated.flatten(order='C'), matrix_spacing):
        ax.text(j[0], j[1], s=round(i,2),horizontalalignment='center',
                 verticalalignment='center');
    
    ax.set_title('Adjacency Matrix for Iteration {}'.format(iteration_number), 
                 fontsize='small',
                 fontweight='semibold');
    
    if row_number!=None:
        rect_row = patches.Rectangle((matrix_col_ticks[0], matrix_row_ticks[row_number+1]),
                                     width=matrix_col_ticks[col_count]-matrix_col_ticks[0],
                                     height=matrix_row_ticks[row_number]-matrix_row_ticks[row_number+1],
                                     facecolor='none',
                                     edgecolor='#1f77b4',
                                     linewidth=2,
                                     clip_on=False)
    
        ax.add_patch(rect_row);    
    
    if col_number!=None:
        rect_col = patches.Rectangle((matrix_col_ticks[col_number+1], matrix_row_ticks[0]),
                                 width=matrix_col_ticks[col_number]-matrix_col_ticks[col_number+1],
                                 height=matrix_row_ticks[row_count]-matrix_row_ticks[0],
                                 facecolor='none',
                                 edgecolor='#ff7f0e',
                                 linewidth=2,
                                 clip_on=False)
    

        ax.add_patch(rect_col);
    
    if cell_idx_list!=None:
        for cell_idx in cell_idx_list:
            rect_cell = patches.Rectangle((matrix_col_ticks[cell_idx[1]+1],matrix_row_ticks[cell_idx[0]+1]),
                                          width=matrix_col_ticks[cell_idx[0]]-matrix_col_ticks[cell_idx[0]+1],
                                          height=matrix_row_ticks[cell_idx[0]]-matrix_row_ticks[cell_idx[0]+1],
                                          facecolor='none',
                                          edgecolor='#2ca02c',
                                          linewidth=2,
                                          clip_on=False)
            ax.add_patch(rect_cell);
    
    return ax

# plot_previousIterationAdjMatrix(G2_adjMatrix, iteration_number=5, row_number=3, col_number=2, ax=ax2)


from IPython.display import SVG, Image
import cairosvg
from io import BytesIO

def plot_iGraphAsSubplot(G, bbox_params, ax):
    originalPlot = igraph.plot(G, bbox=bbox_params)
    pngPlot = BytesIO(cairosvg.svg2png(originalPlot._repr_svg_()))
    pngPlot_read = plt.imread(pngPlot)
    ax.imshow(pngPlot_read, interpolation='spline36');
    ax.axis('off');
    return ax

plot_AdjMatrix(G1_adjMatrix, iteration_number=1, row_number=1, ax=ax1);
plot_iGraphAsSubplot(G1, (200,200), ax2);

plot_AdjMatrix(G1_adjMatrix, iteration_number=3, ax=ax3);
plot_iGraphAsSubplot(G1, (200,200), ax4);

plot_AdjMatrix(G1_adjMatrix, iteration_number=4, cell_idx_list=[(0,2), (3,2)], ax=ax5);
plot_iGraphAsSubplot(G1, (200,200), ax6);


In [ ]:
from ipywidgets import widgets

class SelectMatrixCellButton(widgets.Button):

    def __init__(self, value, *args, **kwargs):
        """Initialize the SelectFilesButton class."""
        super(SelectMatrixCellButton, self).__init__(*args, **kwargs)
        # Create the button.
        self.value = value
        # Set on click behavior.
        self.on_click(self.visualisePathToCell)
    
    @staticmethod #because on_click already passes 'b', or 'self'
    def visualisePathToCell(b):
        display(b.value)

In [ ]:
from ipywidgets import Button, HBox, VBox, Layout

matrix_shape = G1_adjMatrix.shape
display(matrix_shape)
matrix_buttons = [[SelectMatrixCellButton(description=str(cell),
                          value=(row_idx, col_idx),
                         layout=Layout(width='50px', height='50px')) for row_idx, cell in enumerate(col)] for col_idx, col in enumerate(G1_adjMatrix)]

# matrix_buttonsWithValues = [[button_cell(value=col_idx) for button_cell in col] for col_idx, col in enumerate(matrix_buttons)]

# display(matrix_buttonsWithValues)

display(HBox(list(VBox(col) for col in matrix_buttons)))

display(G1_adjMatrix)

def on_button_clicked(b):
    print("Button clicked.")

# HBox.on_click(on_button_clicked)

In [ ]:
display(Markdown(
    r'''

## Making sense of how adjacency matrices change, between G1, G2, G3

We examine how **Node {{row}}** and **Node {{col}}** determine the number of edges at **Iteration {{iteration_number}}**.

Let's observe what changes in matrices:
$$ \begin{bmatrix}1\\0\end{bmatrix} $$

{{vectors[0][3]}}

Got to work out the dictionary bit.

OK, I'll continue tmr. The point is this.
- Each vector represents the connection from that node (i.e. row or col idx) to the other nodes.
- If this vector has a joint node, with another vector (e.g. '1' in the same node), then it shows that in the next iteration, the next hop, it is possible to move from the owner of that vector to that node (which owns the vector).

$$ \require{enclose} \enclose{circle}{x} $$

$$
\begin{bmatrix}
1 & 1 \\
2 & \smash{\fbox{a}} \\
\end{bmatrix}
$$

'''
))

%load_ext itikz



